In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import numpy as np

## Assign ID
1. ID setup: hash(address) + state + zip

In [5]:
def create_id(row):
    addr = '' if pd.isna(row['ADDRESS']) else str(row['ADDRESS']).strip().replace(' ', '').replace(',', '').lower() #address
    s = int(hashlib.sha256((addr).lower().encode('utf-8')).hexdigest(), 16) % 10**4 #only keep last 4 digits
    n1 = 'XX' if pd.isna(row['STATE']) else row['STATE']
    n2 = '00000' if pd.isna(row['ZIP']) else str(int(str(row['ZIP']).split('.')[0].split('-')[0])).zfill(5)
    return str(s).zfill(4) + n1 + n2

2. Check effectiveness of ID

In [6]:
df17geo = pd.read_csv('2017/2017_c_geocoded.csv')
df17geo['ID2'] = df17geo.apply(create_id, axis = 1)
df15geo = pd.read_csv('2015/2015_c_geocoded.csv')
df15geo['ID2'] = df15geo.apply(create_id, axis = 1)

- Single year

In [7]:
print('2017')
print("Number of columns:\t", len(df17geo))
print("Number of unique ID:\t", len(set(df17geo['ID2'])))

2017
Number of columns:	 12031
Number of unique ID:	 11529


In [8]:
#Get a sample of the columns with same ID2
#some services have the same address, others have repeating service with probably different 'keys' column
import collections
counter = collections.Counter(df17geo['ID2'])
dups = [i[0] for i in counter.items() if i[1]>1]
for i in dups[:5]:
    print(df17geo[df17geo['ID2'] == i].iloc[:, 0:5])

                       ID                    ADDRESS     CITY STATE    ZIP
7      Family Life Center  677 West Covington Avenue  Attalla    AL  35954
8  Rapha Treatment Center  677 West Covington Avenue  Attalla    AL  35954
                                     ID          ADDRESS        CITY STATE  \
112  Indian Rivers Mental Health Center  2209 9th Street  Tuscaloosa    AL   
113  Indian Rivers Mental Health Center  2209 9th Street  Tuscaloosa    AL   

       ZIP  
112  35401  
113  35401  
                                     ID                    ADDRESS       CITY  \
124                          Akeela Inc  360 West Benson Boulevard  Anchorage   
125  Akeela Women and Families Programs  360 West Benson Boulevard  Anchorage   

    STATE    ZIP  
124    AK  99503  
125    AK  99503  
                        ID              ADDRESS   CITY STATE    ZIP
255  Community Bridges Inc  5734 East Hope Lane  Globe    AZ  85501
256  Community Bridges Inc  5734 East Hope Lane  Globe    AZ  85

- Comparing two years
    - Major problem: there are addresses with same address but different zip (probably changed/miss recorded)

In [9]:
dups2 = list(df15geo[df15geo['ID2'].isin(set(df17geo['ID2']))]['ID2'])

In [10]:
#most of the duplications should be those with same name
#check the number of facilities w/ different ID same address
#all are of the same address!
for i in dups2[:5]:
    #if (len(df15geo[df15geo['ID2'] == i]) == 1) and (len(df17geo[df17geo['ID2'] == i]) == 1):
    if df15geo[df15geo['ID2'] == i]['ID2'].iloc[0] != df17geo[df17geo['ID2'] == i]['ID2'].iloc[0]:
        print(df15geo[df15geo['ID2'] == i].iloc[:, 0:5])
        print(df17geo[df17geo['ID2'] == i].iloc[:, 0:5], '\n')

In [11]:
#check if there are true negatives of matching
from scipy.spatial import distance
ary = distance.cdist(df15geo.iloc[:, 5:7], df17geo.iloc[:, 5:7], metric='euclidean') #11928*12031=len(df15geo)*len(df17geo)
#check distance distr
#ary[ary < 0.1]
#ary[(ary < 0.1)&(ary > 0.)]

In [17]:
from difflib import SequenceMatcher
def similar(a, b):
    if a == None or b == None:
        return 0
    return SequenceMatcher(None, a, b).ratio()

#check the ones that are very close but with unmatching IDs
for i in range(len(ary))[:300]:
    r = ary[i]
    potential_matches = np.where(r<0.005)[0]
    row = df15geo.iloc[[i]]
    if potential_matches.size != 0:
        matches = df17geo.iloc[potential_matches]
        for index, match in matches.iterrows():
            if row['ID2'].iloc[0] != match['ID2']:
                if similar(row['ADDRESS'].iloc[0], match[1]) > 0.9:
                    print(row.index[0], row['ADDRESS'].iloc[0])
                    print(match.name, match[1], '\n')
                    #check: df15geo.loc[row['ADDRESS'].index[0]], df17geo.loc[match.name]

93 1153 Air Base Boulevard
89 1151 Air Base Boulevard 

94 1151 Air Base Boulevard
88 1153 Air Base Boulevard 

128 360 West Bensen Boulevard
124 360 West Benson Boulevard 

128 360 West Bensen Boulevard
125 360 West Benson Boulevard 

157 3100 South Cushman Street
160 3100 South Cushman Street 

221 1901 North Trekell Road
223 1927 North Trekell Road 

224 1927 North Trekell Road
220 1901 North Trekell Road 

229 562 North Coronodo Boulevard
230 562 North Coronado Boulevard 

266 554 South Bellview Street
276 560 South Bellview Street 

267 554 South Bellview Street
276 560 South Bellview Street 

269 560 South Bellview Street
274 554 South Bellview Street 

269 560 South Bellview Street
275 554 South Bellview Street 

275 619 West Southern Avenue
284 621 West Southern Avenue 

277 621 West Southern Avenue
282 619 West Southern Avenue 



4. Assign ID for other tables

In [18]:
def create_id_raw(row):
    addr = '' if pd.isna(row['Address1']) else str(row['Address1']).strip().replace(' ', '').replace(',', '').lower() #address
    s = int(hashlib.sha256((addr).lower().encode('utf-8')).hexdigest(), 16) % 10**4 #only keep last 4 digits
    n1 = 'XX' if pd.isna(row['State']) else row['State']
    n2 = '00000' if pd.isna(row['ZIP_Code']) else str(int(str(row['ZIP_Code']).split('.')[0].split('-')[0])).zfill(5)
    return str(s).zfill(4) + n1 + n2

In [75]:
#total number of ids
delete_states = ['HI', 'AK', 'PR', 'VI', 'GU', 'AS', 'MP', 'FM', 'PW', 'MH'] #only concerned with mainland
years = [1980, 1990, 1998, 2000, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2015, 2017]
dfs = []
for year in years:
    df = pd.read_csv('{0}/{0}_c.csv'.format(year))
    df[~df['State'].isin(delete_states)].reset_index(drop = True) #remove irrelevant states
    df['ID'] = df.apply(create_id_raw, axis = 1) #add id
    df.to_csv('{0}/{0}_geocode.csv'.format(year), index = False)
    dfs.append(df[['ID', 'Address1', 'City', 'State', 'ZIP_Code']])

In [98]:
#already geocoded IDs
geocoded_id = set(list(df17geo['ID2']) + list(df15geo['ID2']))
len(geocoded_id)

14188

In [109]:
df15geo

,ID,ADDRESS,CITY,STATE,ZIP,Longitude,Latitude,Match Score,ID2
0,SpectraCare,219 Dothan Road,Abbeville,AL,36310,-85.252888,31.553425,100.00,7363AL36310
1,Shelby County Treatment Center,750 Highway 31 South,Alabaster,AL,35007,-86.787385,33.213457,100.00,4041AL35007
2,Lighthouse of Tallapoosa County Inc,36 Franklin Street,Alexander City,AL,35010,-85.950922,32.945359,100.00,6152AL35010
3,South Central Alabama CMHC,205 Academy Drive,Andalusia,AL,36420,-86.446828,31.308620,100.00,1086AL36420
4,South Central Alabama CMHC,205 Academy Drive,Andalusia,AL,36420,-86.446828,31.308620,100.00,1086AL36420
...,...,...,...,...,...,...,...,...,...
11923,Northern Wyoming Mental Health Center,420 1/2 Main Street,Sundance,WY,82729,-104.387628,44.404904,99.55,3383WY82729
11924,Hot Springs County Csl Servs Inc,121 South 4th Street,Thermopolis,WY,82443,-108.208452,43.645239,100.00,5326WY82443
11925,Peak Wellness Center,501 Albany Avenue,Torrington,WY,82240,-104.190737,42.083805,100.00,0027WY82240
11926,Peak Wellness Center,1954 West Mariposa Parkway,Wheatland,WY,82201,-104.969315,42.046687,100.00,5500WY82201


In [143]:
#remaining to be geocoded
df_geo = pd.concat(dfs,ignore_index=True) #dfs is a list
df_geo = df_geo.drop_duplicates(subset='ID', keep='first', ignore_index = True)
df_geo = df_geo[~df_geo['ID'].isin(geocoded_id)]
df_geo.rename(columns={"Address1": "ADDRESS", "City": "CITY", "State":"STATE", "ZIP_Code": "ZIP"}, inplace = True)

In [148]:
def clean_ZIP(row):
    #print(row)
    if not pd.isna(row['ZIP']):
        return str(int(str(row['ZIP']).split('.')[0].split('-')[0]))

df_geo['ZIP'] = df_geo.apply(clean_ZIP, axis = 1)#['ZIP'].astype('str')
#df_geo['ZIP'] = df_geo['ZIP'].apply(lambda x: None if pd.isna(x) else int(x[:5]))
df_geo.to_csv('geocode_master.csv', index = False)

## Match IDs

1. Create master file

In [43]:
d1 = df15geo[['ID2', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'Longitude', 'Latitude', 'Match Score']].rename(columns={'ID2':"ID"})
d2 = df17geo[['ID2', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'Longitude', 'Latitude', 'Match Score']].rename(columns={'ID2':"ID"})
d3 = pd.read_csv("geocode_master.csv")
df_master = pd.concat([d1, d2, d3], ignore_index=True)

In [62]:
df_master = df_master.drop_duplicates('ID', ignore_index=True)

In [67]:
df_master.to_csv("geocode_master.csv", index = False)

2. Filter each table and add match the ID, add long/lat/score

In [65]:
delete_states = ['HI', 'AK', 'PR', 'VI', 'GU', 'AS', 'MP', 'FM', 'PW', 'MH'] #only concerned with mainland
df = pd.read_csv('2000/2000_c.csv')
df[~df['State'].isin(delete_states)].reset_index(drop = True) #remove irrelevant states
df['ID'] = df.apply(create_id_raw, axis = 1)

In [68]:
df_g = df.merge(df_master[['ID', 'Longitude', 'Latitude', 'Match Score']], on = 'ID', how='left')

In [74]:
delete_states = ['HI', 'AK', 'PR', 'VI', 'GU', 'AS', 'MP', 'FM', 'PW', 'MH'] #only concerned with mainland
years = [1980, 1990, 1998, 2000, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2015, 2017]
for year in years:
    df = pd.read_csv('{0}/{0}_c.csv'.format(year))
    df[~df['State'].isin(delete_states)].reset_index(drop = True) #remove irrelevant states
    df['ID'] = df.apply(create_id_raw, axis = 1) 
    df_g = df.merge(df_master[['ID', 'Longitude', 'Latitude', 'Match Score']], on = 'ID', how='left')
    print(year, len(df_g[df_g['Longitude'].isna()]))
    df_g.to_csv('{0}/{0}_g.csv'.format(year), index = False)

1980 0
1990 0
1998 0
2000 0
2003 0
2005 0
2006 0
2007 0
2008 0
2009 0
2010 0
2011 0
2012 0
2015 0
2017 0
